In [38]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import warnings


data_train = pd.read_csv(r"D:\Foundathon\Obesity prediction\train.csv")
data_test = pd.read_csv(r"D:\Foundathon\Obesity prediction\test.csv")

data_train_nans = data_train.dtypes.to_frame().T
data_train_nans['DataFrame'] = 'data train'
data_test_nans = data_test.dtypes.to_frame().T
data_test_nans['DataFrame'] = 'data test'
combined_df = pd.concat([data_train_nans, data_test_nans], axis=0, ignore_index=True)

data_train_nans = data_train.nunique().to_frame().T
data_train_nans['DataFrame'] = 'data train'
data_test_nans = data_test.nunique().to_frame().T
data_test_nans['DataFrame'] = 'data test'
combined_df = pd.concat([data_train_nans, data_test_nans], axis=0, ignore_index=True)

data_train.head()

num_cols_train = list(data_train.select_dtypes(exclude=['object']).columns)
cat_cols_train = list(data_train.select_dtypes(include=['object']).columns)

num_cols_test = list(data_test.select_dtypes(exclude=['object']).columns)
del num_cols_test[0]
cat_cols_test = list(data_test.select_dtypes(include=['object']).columns)

scaler = MinMaxScaler()

data_train[num_cols_train] = scaler.fit_transform(data_train[num_cols_train])
data_test[num_cols_test] = scaler.fit_transform(data_test[num_cols_test])

labelencoder = LabelEncoder()
obj_columns = data_train.select_dtypes(include='object').columns.difference(['NObeyesdad'])

for col_name in obj_columns:
    if data_train[col_name].dtypes == 'object':
        data_train[col_name] = labelencoder.fit_transform(data_train[col_name])

for col_name in data_test.columns:
    if data_test[col_name].dtypes == 'object':
        data_test[col_name] = labelencoder.fit_transform(data_test[col_name])

X = data_train.drop(['NObeyesdad'], axis=1)
y = data_train['NObeyesdad']
y = labelencoder.fit_transform(y)
X_test = data_test

X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size=0.2, random_state=5)

model = DecisionTreeClassifier()

param_grid = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
              'max_depth': [None, 10, 20, 30, 40, 50],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4],
              'max_features': [None, 'sqrt', 'auto', 'log2', 0.5, 0.7]}  # Add numeric values as needed


grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, Y_train)

best_params = grid_search.best_params_

best_model = grid_search.best_estimator_
best_model.fit(X_train, Y_train)

Y_pred = best_model.predict(X_val)
accuracy = accuracy_score(Y_val, Y_pred)
print(f"Accuracy on validation set: {accuracy}")

final_predictions = best_model.predict(X_test)

result_df = pd.DataFrame({'id': range(1, len(final_predictions) + 1), 'NObeyesdad': final_predictions})
result_df['NObeyesdad'] = labelencoder.inverse_transform(result_df['NObeyesdad'])
result_df.to_csv("/kaggle/working/predictions.csv", index=False)


c:\Users\grove\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1080 fits failed out of a total of 6480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1080 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\grove\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\grove\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\grove\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\b

Accuracy on validation set: 0.8728323699421965


In [36]:
import joblib

# Save the model
joblib.dump(best_model, 'obesity_prediction_model.joblib')


['obesity_prediction_model.joblib']

In [37]:
import joblib
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Load the model
best_model = joblib.load('obesity_prediction_model.joblib')

# Sample input features
sample_input = pd.DataFrame({
    'Gender': ['Male'],
    'Age': [25],
    'Height': [1.75],
    'Weight': [70],
    'family_history_with_overweight': ['yes'],
    'FAVC': ['yes'],
    'FCVC': [3],
    'NCP': [3],
    'CAEC': ['Sometimes'],
    'SMOKE': ['no'],
    'CH2O': [2.5],
    'SCC': ['no'],
    'FAF': [1.5],
    'TUE': [2],
    'CALC': ['no'],
    'MTRANS': ['Public_Transportation']
})

# Assuming num_cols_train is the list of numeric columns in your training data
num_cols_train = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

# Assuming data_train is the training dataset
# Exclude the 'id' column from the training data if it exists
data_train = data_train.drop(columns=['id'], errors='ignore')

# Ensure that the columns in the sample input match the order in the training data
sample_input = sample_input[data_train.columns[:-1]]  # Exclude the target column 'NObeyesdad'

# Check if 'id' was in the training data and drop it from the sample input if present
if 'id' in data_train.columns:
    sample_input = sample_input.drop(columns=['id'], errors='ignore')

# Define the scaler, obj_columns, and labelencoder
scaler = MinMaxScaler()
obj_columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
labelencoder = LabelEncoder()

# Preprocess sample input
sample_input[num_cols_train] = scaler.fit_transform(sample_input[num_cols_train])
for col_name in obj_columns:
    if sample_input[col_name].dtypes == 'object':
        sample_input[col_name] = labelencoder.fit_transform(sample_input[col_name])

# Make prediction
prediction = best_model.predict(sample_input)
prediction_class = labelencoder.inverse_transform(prediction)

print(f"Predicted Class: {prediction_class}")


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- id
